In [17]:
import tweepy
import pandas as pd
import numpy as np

In [18]:
tweet_df = pd.read_csv('GOT_tweets_raw.csv', index_col = 0) # index_col to 0 => first column of the dataframe will become the row label
tweet_df.head()

,date,text
0,2019-06-20 10:43:00,RT @acursedlove: EVERYBODY LET’S SAY THANK YOU...
1,2019-06-20 10:38:52,RT @lMMUNlTYIDOL: rip two of the most interest...
2,2019-06-20 10:28:58,Throwback to one of the best frames that ever ...
3,2019-06-20 10:27:51,"Hale and hearty, #KitHarington has reportedly ..."
4,2019-06-20 10:26:08,@dino_melaye Good day twitter sir please help ...


In [19]:
tweet_df.shape

(819, 2)

In [20]:
# Prepare list of texts from Twitter for further analisys.
# 1 - sellect text coloumn and convert it to list list(coloumn) and assign a new var.
# 2 - using list_comprehension or loop preprocess the text removing retweet fragments as RT @thalesgroup
# using f-on split(':') and select string fragment Eg: ab:cd.split(:) will return ['ab', 'cd'].

tweets_texts = list(tweet_df['text'])
tweets_texts = [elem.split(':')[1] for elem in tweets_texts if ':' in elem]
tweets_texts[40]

'//t.co/vzaDAmYL9Y #gameofthrones #GOT #gotchallenge #GameOfThronesFinale https'

In [21]:
tweets_texts = [elem for elem in tweets_texts if '@' not in elem]
tweets_texts[40]

'//t.co/QijeSjWJWi'

In [22]:
# Perform sentiment analisys using textblob library. 
# https://textblob.readthedocs.io/ Najpierw możemy jednak spojrzeć na kilka przykładowych tweetów jaki mają sentyment w Google Cloud Natural Language Processing API. 
# https://cloud.google.com/natural-language/
import random
from textblob import TextBlob

SEED=42
random.seed(SEED)

In [23]:
sample = random.sample(tweets_texts, 30)
for tweet in sample:
    tb = TextBlob(tweet)
    print(tweet)
    print('SENTIMENT: %s' % str(tb.sentiment.polarity)) # "hello %s" % (name,) => hello Ela

 Game of thrones night king and Theon recreation!!!!!! #GameOfThronesFinale #GameofThronesSeason8 #nightking #GameOfThor…
SENTIMENT: -1.0
 Episode 25 is dark and full of spoilers rehashing the finale disappointments of #GamesOfThrones! Listen if you dare #Gam…
SENTIMENT: 0.14375
//t.co/Uz7LgHVbOe
SENTIMENT: 0.0
 NOW AVAILABLE for your listening enjoyment - THE QUAD M SHOW Ep. 156! Download or stream from your favorite podcast catchi…
SENTIMENT: 0.5
 My Game of Thrones Series Finale Review https
SENTIMENT: -0.4
 If Sansa doesn’t show up with a giant Westerosi army I’m gonna be pissed off #GameOfThronesFinale
SENTIMENT: 0.0
 Starks won the game ❤️ #GameOfThronesFinale https
SENTIMENT: -0.4
 YouTuber Rewrites GoT Ending And People Say That This Is How It Should’ve Ended #GameOfThronesFinale
https
SENTIMENT: 0.0
Kings landing #GOTfinal #GameOfTrones #GameOfThonesFinale #gameofthones #GamesOfThrones #JonSnow #DaenerysTargaryen #SansaStark #HBO #AryaStark #GameOfThronesFinale https
SENTIMENT

In [24]:
# Add a coloumn with sentiment to datatframe with tweets
scores = []

for tweet in tweets_texts:
    tb = TextBlob(tweet)
    scores.append(tb.sentiment.polarity)
    
tweet_df['sentiment'] = pd.Series(scores)
tweet_df.head()
print(tweet_df['sentiment'][25:35])

25    0.000000
26   -0.050000
27    0.000000
28    1.000000
29    0.160000
30    0.000000
31    0.000000
32   -0.400000
33    0.000000
34   -0.033333
Name: sentiment, dtype: float64


In [25]:
# Remove all rows containing sentyment == 0.0. Remove duplicates and rows with NaN.

tweet_df.shape
tweet_df.drop_duplicates(inplace=True)

In [26]:
tweet_df = tweet_df[tweet_df['sentiment']!=0.0]
tweet_df.shape

(364, 3)

In [27]:
# Calculate the number of rows with positive and negative sentiment, calculate the percentage

tweet_df[tweet_df['sentiment'] > 0].shape

(230, 3)

In [28]:
tweet_df[tweet_df['sentiment'] < 0].shape

(134, 3)

In [29]:
positive_sent = 230/364*100
negative_sent = 134/364*100
print('positive sentiment = ') 
print(round(positive_sent,2))
print('positive sentiment = ')
print(round(negative_sent,2))

positive sentiment = 
63.19
positive sentiment = 
36.81


In [30]:
# Calculate N of rows with sentyment in estimated borders. 
# Use groupby qnd cut functions: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.cut.html

import numpy as np

tweet_df.groupby(pd.cut(tweet_df['sentiment'], np.arange(-1, 1+0.2, 0.2))).count()

,date,text,sentiment
sentiment,,,
"(-1.0, -0.8]",0,0,0
"(-0.8, -0.6]",0,0,0
"(-0.6, -0.4]",12,12,12
"(-0.4, -0.2]",67,67,67
"(-0.2, -2.22e-16]",41,41,41
"(-2.22e-16, 0.2]",78,78,78
"(0.2, 0.4]",81,81,81
"(0.4, 0.6]",31,31,31
"(0.6, 0.8]",12,12,12


In [15]:
tweet_df

,date,text,sentiment
0,2019-06-20 10:43:00,RT @acursedlove: EVERYBODY LET’S SAY THANK YOU...,1.000000
1,2019-06-20 10:38:52,RT @lMMUNlTYIDOL: rip two of the most interest...,0.162500
5,2019-06-20 10:26:02,RT @lady_ifrit: 🎉🎉🎉🎉The weapon of Arya Stark i...,0.250000
7,2019-06-20 09:28:04,RT @jgamir: Game of Thrones #GameOfThronesFina...,-0.400000
8,2019-06-20 09:11:59,#ThursdayThoughts #GameOfThronesFinale \r\nIt'...,0.437500
14,2019-06-20 06:49:58,Bereket is coming..\r\n#GameOfThronesFinale,0.705000
15,2019-06-20 06:16:28,"Wait, What? #GOT Makers Deleted #CerseiLannist...",1.000000
17,2019-06-20 06:03:03,People Had Incredibly Divided Reactions To The...,0.200000
18,2019-06-20 05:50:11,All of them died for nothing.\r\n#GameOfThrone...,0.333333
21,2019-06-20 05:28:02,RT @laurenceraqq: Power of stories | Tyrion La...,1.000000


In [1]:
tweet_df.shape

NameError: name 'tweet_df' is not defined